In [6]:
#import libraries
!pip install kaggle
import kaggle

#download dataset from kaggle api
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv


Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
#extract file from zip file

import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall() #extract file to dir
zip_ref.close #close file


<bound method ZipFile.close of <zipfile.ZipFile filename='orders.csv.zip' mode='r'>>

In [63]:
#read data from the file and handle null values

import pandas as pd
df = pd.read_csv('orders.csv', na_values = ['Not Available', 'unknown'])
df['Ship Mode'].unique()


array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [65]:
#rename columns

df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.head(5)


,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [67]:
#create new column for discounted price, sale price and profit

df['discount_price'] = df['list_price'] * df['discount_percent']*0.01
df['sale_price'] = df['list_price'] - df['discount_price']
df['profit'] = df['sale_price'] - df['cost_price']


In [69]:
#convert order_date column into a datetime data type

df['order_date']=pd.to_datetime(df['order_date'],format="%Y-%m-%d")

In [71]:
df.drop(columns=['list_price','cost_price','discount_percent'],inplace=True)

In [77]:
#load the data into sql server using replace option

import sqlalchemy as sal
engine = sal.create_engine('mssql://BOOK-9EITCF0R38\\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()


In [82]:
#Load the data into sql server using append option

df.to_sql('df_orders', con = conn, index = False, if_exists = 'replace')

38